In [373]:
import numpy as np
import scipy.optimize as opt
import copy
import matplotlib.pyplot as plt

In [374]:
# Parameters
rho = 0.03
r = 0.05
a = 1
delta = 0.01
c_param = 0.3
l_param = 0.3
B_param = 0.3

param = [rho, r, a, delta, c_param, l_param]

T = 70

In [375]:
# Define utility functions
def u(c, c_param):
    return (c^(1-c_param))/(1-c_param)

def u_prime(c, c_param):
    return c^(-c_param)

def u_prime_inverse(x, c_param):
    return x**(-1/c_param)

def uh(h, l_param):
    return ((1-h)^(1-l_param))/(1-l_param)

def uh_prime(h, l_param):
    return -(1-h)**(-l_param)

def uh_prime_inverse(x, l_param):
    return 1 - ((-x)**(-1/l_param))

In [376]:
# Labor market functions
def g(x):
    return 1.25 - (x*(1.25**0.5 - 0.5) + 0.5)**2

def g_prime_inverse(x):
    return (x/(-2*(1.25**0.5 - 0.5)) - 0.5)/(1.25**0.5 - 0.5)

Create grids

In [377]:
# State grids
na = 30 # number of asset grid points
nk = 20 # number of human capital grid points

amin = 0
amax = 5

kmin = 1
kmax = 5

a_vect = amin + np.linspace(0, 1, na)*(amax - amin) # Vector of A
da = (amax - amin)/(na - 1) # delta in A
a = np.repeat(a_vect[:, None], nk, axis = 1) # Grid of A

k_vect = kmin + np.linspace(0, 1, nk)*(kmax - kmin) # Vector of A
dk = (kmax - kmin)/(nk - 1) # delta in A
k = np.repeat(k_vect[None, :], na, axis = 0) # Grid of K

In [378]:
# Preallocate fwd/bwd derivatives
dVaF = np.zeros((na, nk))
dVaB = np.zeros((na, nk))
dVkF = np.zeros((na, nk))
dVkB = np.zeros((na, nk))

In [379]:
# Time parameters
T = 2
dt = 0.5

In [380]:
# Value function grid
v = np.zeros((na, nk, T))

In [381]:
# Terminal value
small_number1 = 1e-8
small_number2 = 1e-8
aa = np.repeat(a[:, None], nk, axis = 1)
v_terminal = small_number1*((small_number2 + a)**(1-B_param))/(1-B_param)

In [382]:
V = copy.copy(v_terminal)

Compute steady state values

In [401]:
# Steady state requires solving FOC + (drift = 0) = 5 equations, with 5 variables (c, h, k, dVa, dVk)
i = 0
j = 0
dVa0 = np.zeros((na, nk))
dVk0 = np.zeros((na, nk))
C0 = np.zeros((na, nk))
H0 = np.zeros((na, nk))
X0 = np.zeros((na, nk))

In [404]:
def steady_state_err(guess, a, k, param):
    rho, r, a, delta, c_param, l_param = param

    mu_a = h*k*g(x) + r*a - c
    mu_k = h*k*x - delta*k
    focc = u_prime(c, c_param) - dVa
    foch = uh_prime(h, l_param) + dVa*k*g(x) + dVk
    focx = dVa*h*k*g_prime(x) + dVk*h*k

    return mu_a, mu_k, focc, foch, focx

Begin iterations

In [383]:
v[:, :, T-1] = V

In [384]:
# Construct forward difference for a and k
dVaF[0:na-1, :] = (V[1:na, :] - V[0:na-1, :])/da
dVaF[na-1, :] = dVaF[na-2, :]

dVkF[:, 0:nk-1] = (V[:, 1:nk] - V[:, 0:nk-1])/dk
dVkF[:, nk-1] = dVkF[:, nk-2]

dVaB[1:na, :] = (V[1:na, :] - V[0:na-1, :])/da
dVaB[0, :] = dVaB[1, :]

dVkB[:, 1:nk] = (V[:, 1:nk] - V[:, 0:nk-1])/dk
dVkB[:, 0] = dVkB[:, 1]

In [385]:
# Verify value function is increasing in a
if np.sum(dVaF < 0) > 0: # Print message if there exists an element of dVaF < 0
    print('V is not monotonically increasing in a')

if np.sum(dVkF < 0) > 0: # Print message if there exists an element of dVkF < 0
    print('V is not monotonically increasing in k')

In [386]:
# FOC for consumption, which depends only on dVa
cF = u_prime_inverse(dVaF, c_param)
cB = u_prime_inverse(dVaB, c_param)

In [387]:
# FOC for x, which depends only on dVa and dVk. Corners at 0 and 1 taken into account.
xFF = np.clip(g_prime_inverse(-dVkF/dVaF), 0, 1)
xFB = np.clip(g_prime_inverse(-dVkB/dVaF), 0, 1)
xBF = np.clip(g_prime_inverse(-dVkF/dVaB), 0, 1)
xBB = np.clip(g_prime_inverse(-dVkB/dVaB), 0, 1)

In [388]:
# FOC for h. Corners at 0 and 1 taken into account.
hFF = np.clip(uh_prime_inverse(-dVaF*k*g(xFF) - dVkF, l_param), 0, 1)
hFB = np.clip(uh_prime_inverse(-dVaF*k*g(xFB) - dVkB, l_param), 0, 1)
hBF = np.clip(uh_prime_inverse(-dVaB*k*g(xBF) - dVkF, l_param), 0, 1)
hBB = np.clip(uh_prime_inverse(-dVaB*k*g(xBB) - dVkB, l_param), 0, 1)

In [389]:
# Drift matrices for a
mu_aFF = hFF*k*g(xFF) + r*a - cF
mu_aFB = hFB*k*g(xFB) + r*a - cF
mu_aBF = hBF*k*g(xBF) + r*a - cB
mu_aBB = hBB*k*g(xBB) + r*a - cB

# Drift matrices for k
mu_kFF = hFF*k*xFF - delta*k
mu_kFB = hFB*k*xFB - delta*k
mu_kBF = hBF*k*xBF - delta*k
mu_kBB = hBB*k*xBB - delta*k

In [390]:
# Create indicator matrices indicating consistency of direction of drift with fwd/bwd difference
# Impose False at boundary if fwd/bwd difference implies drift sends state beyond boundary
I_FF = np.logical_and(mu_aFF > 0, mu_kFF > 0)
I_FF[na-1, :] = False
I_FF[:, nk-1] = False

I_FB = np.logical_and(mu_aFB > 0, mu_kFB < 0)
I_FB[na-1, :] = False
I_FB[:, 0] = False

I_BF = np.logical_and(mu_aBF < 0, mu_kFF > 0)
I_BF[0, :] = False
I_BF[:, nk-1] = False

I_BB = np.logical_and(mu_aBB < 0, mu_kFB < 0)
I_BB[0, :] = False
I_BB[:, 0] = False

In [391]:
# Compute validity of fwd/bwd difference combinations
# Stack all indicator matrices
I_stacked = np.zeros((na, nk, 4))
I_stacked[:, :, 0] = I_FF
I_stacked[:, :, 1] = I_FB
I_stacked[:, :, 2] = I_BF
I_stacked[:, :, 3] = I_BB

# Use logical_or over all 4 combinations; if at least one is valid, I_valid is True
I_valid = np.logical_or.reduce((I_FF, I_FB, I_BF, I_BB))
I_type = np.argmax(I_stacked, axis = 2)

In [392]:
# Function computes drift in a given a dVa
def boundary_dVa(dVa, dVkj, a, kj, l_param):
    xstar = np.clip(g_prime_inverse(-dVkj/dVa), 0, 1)
    hstar = np.clip(uh_prime_inverse(-dVa*kj*g(xstar) - dVkj, l_param), 0, 1)
    return hstar*kj*g(xstar) + r*a - dVa**(-1/c_param)

In [393]:
# Impose state boundary condition for a_min
# First look at the case with mu_kBF
# Loop over a_min (upper edge) in the j dimension
aind = 0
for j in range(0, nk):
    if I_valid[aind, j] == False:
        dVkj = dVkF[aind, j]
        aval = a[aind, 0]
        kj = k[aind, j]

        # Find dVaB that satisfies FOC and a_min boundary conditio
        dVa_guess = dVaB[aind, j]
        dVaj = opt.fsolve(boundary_dVa, dVa_guess, args = (dVkj, aval, kj, l_param))[0]

        # Compute drift and evaluate if drift is consistent with chosen fwd diff for k
        xstar = np.clip(g_prime_inverse(-dVkj/dVaj), 0, 1)
        hstar = np.clip(uh_prime_inverse(-dVaj*kj*g(xstar) - dVkj, l_param), 0, 1)
        cstar = u_prime_inverse(dVaj, c_param)

        mu_aj = hstar*kj*g(xstar) + r*aval - cstar
        mu_kj = hstar*kj*xstar - delta*kj

        if mu_kj >= 0 and j != nk - 1:
            print('BF scheme success')

            # fwd diff for k is successful; BF scheme works
            I_valid[aind, j] = True
            I_type[aind, j] = 2

            # Replace bwd diff approximation for aval at j
            dVaB[aind, j] = dVaj

            # Replace bwd diff drift values
            mu_aBF[aind, j] = mu_aj
            mu_kBF[aind, j] = mu_kj

        # BF scheme failed; try BB scheme
        else:
            print('BF scheme failed; now trying BB scheme')
            dVkj = dVkB[aind, j]
            dVaj = opt.fsolve(boundary_dVa, dVa_guess, args = (dVkj, aval, kj, l_param))[0]

            xstar = np.clip(g_prime_inverse(-dVkj/dVaj), 0, 1)
            hstar = np.clip(uh_prime_inverse(-dVaj*kj*g(xstar) - dVkj, l_param), 0, 1)
            cstar = u_prime_inverse(dVaj, c_param)

            # Evaluate if drift is consistent with chosen bwd diff for k
            mu_aj = hstar*kj*g(xstar) + r*aval - cstar
            mu_kj = hstar*kj*xstar - delta*kj


            if mu_kj <= 0 and j != 0:
                print('BB scheme success')
                # bwd diff for k is successful; BB scheme works
                I_valid[aind, j] = True
                I_type[aind, j] = 3

                # Replace bwd diff approximation for aval at j
                dVaB[aind, j] = dVaj

                # Replace bwd diff drift values
                mu_aBB[aind, j] = mu_aj
                mu_kBB[aind, j] = mu_kj

BF scheme failed; now trying BB scheme
BF scheme failed; now trying BB scheme
BB scheme success
BF scheme failed; now trying BB scheme
BB scheme success
BF scheme failed; now trying BB scheme
BB scheme success
BF scheme failed; now trying BB scheme
BB scheme success
BF scheme failed; now trying BB scheme
BB scheme success
BF scheme failed; now trying BB scheme
BB scheme success
BF scheme failed; now trying BB scheme
BB scheme success
BF scheme failed; now trying BB scheme
BB scheme success
BF scheme failed; now trying BB scheme
BB scheme success
BF scheme failed; now trying BB scheme
BB scheme success
BF scheme failed; now trying BB scheme
BB scheme success
BF scheme failed; now trying BB scheme
BB scheme success
BF scheme failed; now trying BB scheme
BB scheme success
BF scheme failed; now trying BB scheme
BB scheme success
BF scheme failed; now trying BB scheme
BB scheme success
BF scheme failed; now trying BB scheme
BB scheme success
BF scheme failed; now trying BB scheme
BB scheme 

In [394]:
# Impose state boundary condition for a_max
# First look at the case with mu_kBF
# Loop over a_max (upper edge) in the j dimension
aind = na-1
for j in range(0, nk):
    if I_valid[aind, j] == False:
        dVkj = dVkF[aind, j]
        aval = a[aind, 0]
        kj = k[aind, j]

        # Find dVaF that satisfies FOC and a_max boundary condition
        dVa_guess = dVaF[aind, j]
        dVaj = opt.fsolve(boundary_dVa, dVa_guess, args = (dVkj, aval, kj, l_param))[0]

        # Compute drift and evaluate if drift is consistent with chosen fwd diff for k
        xstar = np.clip(g_prime_inverse(-dVkj/dVaj), 0, 1)
        hstar = np.clip(uh_prime_inverse(-dVaj*kj*g(xstar) - dVkj, l_param), 0, 1)
        cstar = u_prime_inverse(dVaj, c_param)

        mu_aj = hstar*kj*g(xstar) + r*aval - cstar
        mu_kj = hstar*kj*xstar - delta*kj

        if mu_kj >= 0 and j != nk - 1:
            print('FF scheme success')
            # fwd diff for k is successful; FF scheme works
            I_valid[aind, j] = True
            I_type[aind, j] = 0

            # Replace fwd diff approximation for aval at j
            dVaF[aind, j] = dVaj

            # Replace bwd diff drift values
            mu_aFF[aind, j] = mu_aj
            mu_kFF[aind, j] = mu_kj

        # BF scheme failed; try BB scheme
        else:
            print('FF scheme failed; now trying FB scheme')
            dVkj = dVkB[aind, j]
            dVaj = opt.fsolve(boundary_dVa, dVa_guess, args = (dVkj, aval, kj, l_param))[0]

            xstar = np.clip(g_prime_inverse(-dVkj/dVaj), 0, 1)
            hstar = np.clip(uh_prime_inverse(-dVaj*kj*g(xstar) - dVkj, l_param), 0, 1)
            cstar = u_prime_inverse(dVaj, c_param)

            # Evaluate if drift is consistent with chosen bwd diff for k
            mu_aj = hstar*kj*g(xstar) + r*aval - cstar
            mu_kj = hstar*kj*xstar - delta*kj

            if mu_kj <= 0 and j != 0:
                print('FB scheme success')
                # bwd diff for k is successful; BB scheme works
                I_valid[aind, j] = True
                I_type[aind, j] = 1

                # Replace bwd diff approximation for aval at j
                dVaB[aind, j] = dVaj

                # Replace bwd diff drift values
                mu_aFB[aind, j] = mu_aj
                mu_kFB[aind, j] = mu_kj

FF scheme failed; now trying FB scheme


In [395]:
# Function computes drift in k for a given dVk
def boundary_dVk(dVk, dVai, ai, k, l_param):
        xstar = np.clip(g_prime_inverse(-dVk/dVai), 0, 1)
        hstar = np.clip(uh_prime_inverse(-dVai*k*g(xstar) - dVk, l_param), 0, 1)
        return hstar*k*xstar - delta*k

In [396]:
# Impose state boundary condition for k_min
# First look at the case with mu_aFB
# Loop over k_min (upper edge) in the i dimension
kind = 0
for i in range(0, na):
    if I_valid[i, kind] == False:
        dVai = dVaF[i, kind]
        kval = k[0, kind]
        ai = a[i, kind]

        # Find dVkB that satisfies FOC and k_min boundary condition
        dVk_guess = dVkF[i, kind]
        dVki = opt.fsolve(boundary_dVk, dVk_guess, args = (dVai, ai, kval, l_param))[0]

        # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
        xstar = np.clip(g_prime_inverse(-dVki/dVai), 0, 1)
        hstar = np.clip(uh_prime_inverse(-dVai*kval*g(xstar) - dVki, l_param), 0, 1)
        cstar = u_prime_inverse(dVai, c_param)

        mu_ai = hstar*kval*g(xstar) + r*ai - cstar
        mu_ki = hstar*kval*xstar - delta*kval

        if mu_ai >= 0 and i != na:
            print('FB scheme success')
            # fwd diff for a is successful; FB scheme works
            I_valid[i, kind] = True
            I_type[i, kind] = 1

            # Replace bwd diff approximation for kmin at i
            dVkB[i, kind] = dVki

            # Replace bwd diff drift values
            mu_aFB[i, kind] = mu_ai
            mu_kFB[i, kind] = mu_ki

        # FB scheme failed, try BB
        else:
            print('FB scheme failed; now trying BB scheme')
            dVai = dVaB[i, kind]

            # Find dVkB that satisfies FOC and k_min boundary condition
            dVk_guess = dVkB[i, kind]
            dVki = opt.fsolve(boundary_dVk, dVk_guess, args = (dVai, ai, kval, l_param))[0]

            # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
            xstar = np.clip(g_prime_inverse(-dVki/dVai), 0, 1)
            hstar = np.clip(uh_prime_inverse(-dVai*kval*g(xstar) - dVki, l_param), 0, 1)
            cstar = u_prime_inverse(dVai, c_param)

            mu_ai = hstar*kval*g(xstar) + r*ai - cstar
            mu_ki = hstar*kval*xstar - delta*kval

            if mu_ai <= 0 and i != 0:
                print('BB scheme success')
                # bwd diff for a is successful; BB scheme works
                I_valid[i, kind] = True
                I_type[i, kind] = 3

                # Replace bwd diff approximation for amax at j
                dVkB[i, kind] = dVki

                # Replace bwd diff drift values
                mu_aBB[i, kind] = mu_ai
                mu_kBB[i, kind] = mu_ki


FB scheme failed; now trying BB scheme
FB scheme failed; now trying BB scheme
BB scheme success
FB scheme failed; now trying BB scheme
BB scheme success
FB scheme failed; now trying BB scheme
BB scheme success
FB scheme failed; now trying BB scheme
BB scheme success
FB scheme failed; now trying BB scheme
BB scheme success
FB scheme failed; now trying BB scheme
BB scheme success
FB scheme failed; now trying BB scheme
BB scheme success
FB scheme failed; now trying BB scheme
BB scheme success
FB scheme failed; now trying BB scheme
BB scheme success
FB scheme failed; now trying BB scheme
BB scheme success
FB scheme failed; now trying BB scheme
BB scheme success
FB scheme failed; now trying BB scheme
BB scheme success
FB scheme failed; now trying BB scheme
BB scheme success
FB scheme failed; now trying BB scheme
BB scheme success
FB scheme failed; now trying BB scheme
BB scheme success
FB scheme failed; now trying BB scheme
BB scheme success
FB scheme failed; now trying BB scheme
BB scheme 

In [397]:
# Impose state boundary condition for k_max
# First look at the case with mu_aFF
# Loop over k_max (upper edge) in the i dimension
kind = 0
for i in range(0, na):
    if I_valid[i, kind] == False:
        dVai = dVaF[i, kind]
        kval = k[0, kind]
        ai = a[i, kind]

        # Find dVkF that satisfies FOC and k_max boundary condition
        dVk_guess = dVkB[i, kind]
        dVki = opt.fsolve(boundary_dVk, dVk_guess, args = (dVai, ai, kval, l_param))[0]

        # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
        xstar = np.clip(g_prime_inverse(-dVki/dVai), 0, 1)
        hstar = np.clip(uh_prime_inverse(-dVai*kval*g(xstar) - dVki, l_param), 0, 1)
        cstar = u_prime_inverse(dVai, c_param)

        mu_ai = hstar*kval*g(xstar) + r*ai - cstar
        mu_ki = hstar*kval*xstar - delta*kval

        if mu_ai >= 0 and i != na:
            print('FF scheme success')
            # fwd diff for a is successful; FF scheme works
            I_valid[i, kind] = True
            I_type[i, kind] = 0

            # Replace fwd diff approximation for kmax at i
            dVkF[i, kind] = dVki

            # Replace drift values
            mu_aFF[i, kind] = mu_ai
            mu_kFF[i, kind] = mu_ki

        # FF scheme failed, try BF
        else:
            print('FF scheme failed; now trying BF scheme')
            dVai = dVaB[i, kind]

            # Find dVkB that satisfies FOC and k_max boundary condition
            dVk_guess = dVkF[i, kind]
            dVki = opt.fsolve(boundary_dVk, dVk_guess, args = (dVai, ai, kval, l_param))[0]

            # Compute drift and evaluate if drift is consistent with chosen fwd diff for a
            xstar = np.clip(g_prime_inverse(-dVki/dVai), 0, 1)
            hstar = np.clip(uh_prime_inverse(-dVai*kval*g(xstar) - dVki, l_param), 0, 1)
            cstar = u_prime_inverse(dVai, c_param)

            mu_ai = hstar*kval*g(xstar) + r*ai - cstar
            mu_ki = hstar*kval*xstar - delta*kval

            if mu_ai <= 0 and i != 0:
                print('BF scheme success')
                # bwd diff for a is successful; BF scheme works
                I_valid[i, kind] = True
                I_type[i, kind] = 2

                # Replace fwd diff approximation for kmax at i
                dVkF[i, kind] = dVki

                # Replace bwd diff drift values
                mu_aBF[i, kind] = mu_ai
                mu_kBF[i, kind] = mu_ki


FF scheme failed; now trying BF scheme
